# Data Analysis

This are charts to illustrate the behavior of different curves under the effect of variation of vanishing policies. 

In [ ]:
import symupy
import pandas as pd
import numpy as np 
import os
import sys
import re
from itertools import repeat, chain
import ipywidgets as widgets
from IPython.display import display
import networkx as nx
import matplotlib.pyplot as plt 

# try:
#     pd.set_option('plotting.backend', 'hvplot')
# except: 
#     pass
print(f"Backend: {pd.options.plotting.backend}")

from symupy.api import Simulator,Simulation

print(f"Version of symupy: {symupy.__version__}")

packages = ['src', 'data/results/']

# Adding supplementary functions 
for pck in packages:
    print(f"Adding folder: {pck}")
    sys.path.append(os.path.join(os.getcwd(),f"../{pck}"))


In [ ]:
# ==============
# Widgets
# ==============

zones_cpt = [f'Cpt_{x}' for x in list(np.arange(0,9)+1)]

zonewdgt = widgets.RadioButtons(
    options = zones_cpt,
    value=zones_cpt[4], # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Zone:',
    disabled=False
)
prc_sel = widgets.SelectMultiple(
    options=['All',0,10,20,30,40,50,70],
    value=['All'],
    #rows=10,
    description='Vanishing Policy',
    disabled=False
)

In [ ]:
# ==============
# Plot functions 
# ==============

def load_data(zone = 'Cpt_5'):
    """Reading and loading files"""
    
    output_dir = os.listdir('../data/results')
    output_dir = sorted(output_dir) # Picking good cases
    output_dir = [x for x in output_dir if x.startswith(zone)]

    full_ttt = []
    full_ttd = []
    full_spd = []
    full_ctr = []
    for case in output_dir:
        load_dir = os.getcwd()+'/../data/results/'+ case+'/'
        
        filettt = load_dir +'ttt.csv'
        filettd = load_dir +'ttd.csv'
        filespd = load_dir +'spd.csv'
        filectr = load_dir +'ctr.csv'
        
        ttt = pd.read_csv(filettt)        
        ttd = pd.read_csv(filettd)
        spd = pd.read_csv(filespd)
        ctr = pd.read_csv(filectr)
        
        full_ttt.append(ttt)
        full_ttd.append(ttd)
        full_spd.append(spd)
        full_ctr.append(ctr)        

    key_prc = [0,10,20,30,40,50,70]
    dfttt = pd.concat(full_ttt, keys = key_prc, axis = 0)
    dfttd = pd.concat(full_ttd, keys = key_prc, axis = 0)
    dfspd = pd.concat(full_spd, keys = key_prc, axis = 0)
    dfctr = pd.concat(full_ctr, keys = key_prc, axis = 0)
    return dfttt,dfttd, dfspd,dfctr 

def filter_data(var, zone='Cpt_5'):
    return var.unstack(level=0)[zone]

data = load_data() 

def plot_data(zone='Cpt_5',prc = None):
    data_flt = [filter_data(var) for var in data]        
    df=pd.concat(data_flt,keys = ['TTT','TTD','SPD','CTR'])

    print(prc,zone)
    
    if prc[0]=="All" or prc is None:
        df = df.reset_index(level=0);
    else:
        df = df.loc[:,prc] 
        df = df.reset_index(level=0)        
  
    for title, group in df.groupby('level_0'):
        group.plot(title=title, grid = True, figsize=(15,5))

   

In [ ]:
data_view = widgets.interactive(plot_data,zone=zonewdgt, prc=prc_sel)
zonewdgt.observe(lambda x: data_view.update(), 'value')
display(data_view)

## Routing Data 

We analize here the number of reroutings per case 

In [ ]:
## Rerouting data 

def load_data_routing(zone='Cpt_5'):
    output_dir = os.listdir('../data/results')
    output_dir = sorted(output_dir) # Picking good cases
    output_dir = [x for x in output_dir if x.startswith(zone)]

    lst_route = []
    for case in output_dir:
        load_dir = os.getcwd()+'/../data/results/'+ case+'/OUT/defaultOut_ctrlzonedata_.csv'
        routing = pd.read_csv(load_dir)
        lst_route.append(routing)
    return lst_route

lst_route = load_data_routing()
key_prc = [0,10,20,30,40,50,70]
dfrt = pd.concat(lst_route, keys= key_prc,axis=0)
dfrt = dfrt.reset_index(level=0)
dfrt.rename(columns={'level_0': 'Vanishing %'}, inplace=True)
dfrt.groupby('Vanishing %').sum()['activation'].plot(kind='bar',title='Number of Activations', grid = True, figsize = (7,5));

Number of activations distributed per zone:

In [ ]:
dfgrp = dfrt.groupby(['zone','Vanishing %',]).sum()
dfgrp['activation'].unstack(level='zone')

In [ ]:
delay = dfrt.groupby(['Vanishing %','zone','veh']).agg({'time': lambda x: x.max() - x.min()})

In [ ]:
delay_plot = delay.reset_index(level='veh').groupby(['Vanishing %','zone']).agg({'time':['mean','std']})

#### Mean average activation deactivation time difference 

In [ ]:
delay_plot.unstack(level='zone')[('time','mean')].plot(kind='bar', figsize= (15,5), grid=True);

#### Standard deviation activation dactivation time difference

In [ ]:
delay_plot.unstack(level='zone')[('time','std')].plot(kind='bar' , figsize= (15,5), grid=True);

#### Zone 5: Time difference between activation and desactivation

In [ ]:
# ==============
# Widgets
# ==============

zone2wdgt = widgets.RadioButtons(
    options = [331,332,333,334,335,336,337,338,339],
    value= 335, # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Zone:',
    disabled=False
)

prc2wdgt = widgets.RadioButtons(
    options = [0,10,20,30,40,50,70],
    value=20, # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Zone:',
    disabled=False
)

# ==============
# Plot functions 
# ==============

def plot_time_activation(prc=10,zone=335):
    ax = delay.loc[(prc,zone ,slice(None)),:].reset_index(level='veh')['time'].plot(kind='hist',bins = 40, density=True, grid = True, figsize= (15,5), title = f"Distribution for {prc} % in zone {zone}");
    ax.set_xlabel("Time between activation deactivation");
    


In [ ]:
data2_view = widgets.interactive(plot_time_activation,zone=zone2wdgt, prc=prc2wdgt)
zone2wdgt.observe(lambda x: data2_view.update(), 'value')
prc2wdgt.observe(lambda x: data2_view.update(), 'value')
display(data2_view)

### Proposed control algorithm 

For the controlled tests the following are two proposition under tests: 

Consider the following model where $x_i$ represents the state of a single region, in this case the aggregated speed of a region. Let's supose a model where the change in speed can be altered by a vanishing control given in the form of $u_i$ for a specific region as. The continuous and discrete counter parts of this model are represented as: 

$$
\dot{x} = u \rightarrow x(k+1) = x(k) + \varepsilon u(k)
$$

Consider now that the region belongs to a network of agents where the edges can be traced to ***adjacent physical*** meaning adjacent physical regions. In this sense a network of agents can be created via a simple undirected graph such as: 

In [ ]:
from algorithms.control import define_grid_graph
G = define_grid_graph(3, 3)
nx.draw(G, node_color="#A0CBE2", with_labels=True)
color_map = []
for node in G:
    if node in  nx.neighbors(G,'Cpt_5') or node=='Cpt_5':
        color_map.append('Blue')
    else:
        color_map.append("#A0CBE2")

A define set of neighbors for the zone 5 would be in this case for example 

In [ ]:
nx.draw(G, node_color=color_map, with_labels=True)

One **static control** under test is the following:

Let consider a regulator for a single region as a proportion of the error between the current state and the desired state: 

$$
r(k) =  k_i \left(\frac{\bar{x}_i-x_i(k)}{\bar{x}_i}\right)
$$

In this case, the vanishing policy will increase with respect to the error in a proportional way determined by $k_i$. It is already known that the vanishing policy should not act in the free-flow region, therefore we limit its action by introducing a piecewise behavior 

$$
r(k) =  k_i \min\left(\frac{\bar{x}_i-x_i(k)}{\bar{x}_i},0\right)
$$

The principle is the same as an activation function of a specific neural network. Although up to the moment it sounds a reasonable strategy the idea just considers the fact that information is locally acting. 

#### Cooperation effect 

The cooperation effect proposed for this approach is to consider neighbors local controls at current time instant in order to contribute to the current policy. Let consider an agent $i$ with a and its set of agents contained in a set $\mathcal{N}_i$. 

The proposed control law consists on considering this individual contribution and add possible effect of differences among normalized states, via an instantaneous proportional - integral action in this way:

Let's introduce the effect of normalization via $z_i = \frac{x_i}{\bar{x}_i}$

***Proportional version***
$$
u_i(k) =  k_j \min\left(1-z_i(k),0\right)+ \alpha(k) \sum_{j \in \mathcal{N}_i} \left(z_i(k)-z_j(k)\right)   ,\quad 0<k_i\leq 1
$$

***Proportional-integral variation***
$$
u_i(k) =  k_j \min\left(1-z_i(k),0\right)+  \sum_{j \in \mathcal{N}_i}  \left( \alpha(k) \left(z_i(k) - z_j(k)\right) +  \beta(k)\sum_{\kappa=0}^{k} \left(z_i(k)-z_j(k)\right) \right)  ,\quad 0<k_i\leq 1
$$

#### Advantages

* Self control for the region 
* Normalization for the computation of the vanishing policy 
* Cooperation is introduced via diferences of normalized states 
* Depends on calibration of values $\bar{x}_i$

#### Disadvantages 

* The protocol can be instanteneous for the moment for simplicity 
* Tuning of $\alpha$ can be done on line but assumes another conditions 
* The control does not acknowledge any information on the demand

Andres L.